\begin{center}
Gabe Morris
\end{center}

In [1]:
# Imports
import numpy as np
from scipy.optimize import fsolve
from msu_esd import Pipe

\pagebreak
\tableofcontents
\pagebreak

# Given
From Project 1, piping and pump requirements for a chilled fluid circulating system for a portion of the Orlando Airport were determined. A Z-network was used for the system, and the pumps head and flow rate requirements were found.

## Project 1 Overview
From Project 1, the booster pump option was selected because it was more cost-efficient and offered more control over the system. The schematic is shown in Figure 1.

\begin{center}
\includegraphics{images/fig1} \\
Figure 1: Booster Pump Schematic
\end{center}

The resulting head values are $b_2=67.65\,ft$, $b_4=101.95\,ft$, $b_7=55.7\,ft$, and $b_{10}=44.9\,{ft}$. The main line pump has a head of $W_s=145\,ft$. The summary of the piping lengths as well as material properties are given in Table 1.

\begin{center}
Table 1: Summary of Piping
\end{center}

| Pipe | L (ft) | D (in) | # Valves | # Elbows | K    | C   |
|------|--------|--------|----------|----------|------|-----|
| 1    | 2840   | 18.812 | 1        | 0        | 1.78 | 8   |
| 2    | 2380   | 10.02  | 1        | 5        | 5.53 | 8   |
| 3    | 1300   | 11.938 | 1        | 0        | 1.78 | 8   |
| 4    | 1630   | 10.02  | 1        | 5        | 5.53 | 8   |
| 5    | 3000   | 11.938 | 1        | 2        | 3.28 | 8   |
| 6    | 5000   | 11.938 | 1        | 2        | 3.28 | 8   |
| 7    | 1580   | 11.938 | 1        | 0        | 1.78 | 8   |
| 8    | 1550   | 7.981  | 1        | 5        | 5.53 | 8   |
| 9    | 1550   | 7.981  | 1        | 7        | 7.03 | 8   |
| 10   | 5130   | 10.02  | 1        | 8        | 7.78 | 8   |
| 11   | 1875   | 11.938 | 1        | 1        | 2.53 | 8   |

\begin{center}
Material: Galvanized Steel (Schedule 40) \\
Absolute Roughness: $\epsilon=0.0005\,ft$ \\
Working Fluid: Therminol D-12 \\
Density: $\rho=48.05\frac{lbm}{ft^3}$ \\
Heat Capacity: $c_p=0.491\frac{Btu}{lbm^\circ F}$ \\
Viscosity: $\mu=3.715\frac{lbm}{ft\,hr}$
\end{center}

The system of equations using the Kirchhoff method is,

$$\begin{cases}
Q_1=Q_3+Q_2 \\
Q_3=Q_4+Q_5 \\
Q_6=Q_2+Q_4 \\
Q_5=Q_7+Q_{10} \\
Q_7=Q_8+Q_9 \\
Q_{11}=Q_6+Q_7 \\
h_2+K_1Q_2|Q_2|+h_6+h_{11}+h_1-150+0.1Q_1|Q_1|-b_2=0 \\
h_4+K_3Q_4|Q_4|-h_2-K_1Q_2|Q_2|+h_3+b_2-b_4=0 \\
h_7+h_8+K_4Q_8|Q_8|-h_6-h_4-K_3Q_4|Q_4|+h_5+b_4-b_7=0 \\
h_9+K_4Q_9|Q_9|-h_8-K_4Q_8|Q_8|=0 \\
h_{10}+K_2Q_{10}|Q_{10}|-h_{11}-h_7-h_9-K_4Q_9|Q_9|+b_7-b_{10}=0
\end{cases}$$

The code block below outputs the flow rates for this system.

In [2]:
# Define known constants
K1, K2, K3, K4 = 4.5, 4.5, 4.5, 10  # Loss coefficients

rho = 48.05/32.174  # In slugs per cubic feet
mu = 3.715/(3600*32.174)  # In slugs per (ft s) or lbf*s per ft squared
epsilon = 0.0005  # In ft

D20, D12, D10, D8 = np.array([18.812, 11.938, 10.02, 7.981])/12  # Diameters in ft

Ws = 145  # in ft

# Define pipe objects
p1 = Pipe(D20, 2840, epsilon, rho, mu, K=1.78, C=8)
p2 = Pipe(D10, 2380, epsilon, rho, mu, K=5.53, C=8)
p3 = Pipe(D12, 1300, epsilon, rho, mu, K=1.78, C=8)
p4 = Pipe(D10, 1630, epsilon, rho, mu, K=5.53, C=8)
p5 = Pipe(D12, 3000, epsilon, rho, mu, K=3.28, C=8)
p6 = Pipe(D12, 5000, epsilon, rho, mu, K=3.28, C=8)
p7 = Pipe(D12, 1580, epsilon, rho, mu, K=1.78, C=8)
p8 = Pipe(D8, 1550, epsilon, rho, mu, K=5.53, C=8)
p9 = Pipe(D8, 1550, epsilon, rho, mu, K=7.03, C=8)
p10 = Pipe(D10, 5130, epsilon, rho, mu, K=7.78, C=8)
p11 = Pipe(D12, 1875, epsilon, rho, mu, K=2.53, C=8)

def balanced(x, b2, b4, b7, b10):
    Q1, Q2, Q3, Q4, Q5, Q6, Q7, Q8, Q9, Q10, Q11 = x
    # all expressions need to be set to zero
    return [
        Q1 - Q2 - Q3,
        Q6 - Q2 - Q4,
        Q3 - Q4 - Q5,
        Q5 - Q10 - Q7,
        Q7 - Q8 - Q9,
        Q11 - Q6 - Q7,
        K1*Q2*abs(Q2) + p2.h(Q2) + p6.h(Q6) + p11.h(Q11) + p1.h(Q1) - Ws + 0.1*Q1*abs(Q1) - b2,
        p4.h(Q4) + K3*Q4*abs(Q4) - p2.h(Q2) - K1*Q2*abs(Q2) + p3.h(Q3) + b2 - b4,
        p7.h(Q7) + p8.h(Q8) + K4*Q8*abs(Q8) - p6.h(Q6) - p4.h(Q4) - K3*Q4*abs(Q4) + p5.h(Q5) + b4 - b7,
        p9.h(Q9) + K4*Q9*abs(Q9) - p8.h(Q8) - K4*Q8*abs(Q8),
        p10.h(Q10) + K2*Q10*abs(Q10) - p11.h(Q11) - p9.h(Q9) - K4*Q9*abs(Q9) - p7.h(Q7) + b7 - b10
    ]

Q_guess = np.array([5, 1, 4, 2, 2, 3, 1, 3, -2, 1, 4])  # This does satisfy mass conservation
b = 67.65, 101.95, 55.7, 44.9
balanced_solution = fsolve(balanced, Q_guess, args=(*b, ))
balanced_solution  # flow rates in ft^3/s

array([10.69253924,  2.71300891,  7.97953032,  2.7132475 ,  5.26628283,
        5.42625641,  2.55233847,  1.27984524,  1.27249323,  2.71394436,
        7.97859488])

# Find
Using the results from Project 1 and the Goulds handout, accomplish the following:

a. Are viscous fluid pump corrections needed?
b. Select the required main-line pump (company, model number, RPM, rotor diameter). A copy of the manufacturer’s pump performance information is needed in the report. If booster pumps are used, then continue to use the same booster pumps increases in head as in Project 1 (the corrected version), but you do not need to specify an exact model.
c. Generate a curve fit for H vs. Q for the main-line pump of part (b). Verify using the Hardy-Cross or Kirchhoff program that the selected pump is suitable. This means that the H-Q curve fit is to be placed in the Hardy-Cross or Kirchhoff program and the resulting flow rates in the individual loads are to be computed and compared with the required flow rates. Compute the power required for the pump including viscous corrections, if needed.
d. The management team also wants a parallel pump arrangement, using identical pumps, for the main line to be examined. Booster pumps, if used, are the same as in Part (b). Select the required pump (model number, RPM, rotor diameter). A copy of the manufacturer’s pump performance information is needed. Use the Hardy-Cross or Kirchhoff program to find the behavior of the system with (1) both pumps in parallel and (2) only one of the parallel pumps in operation. This gives some idea of the system capability of the parallel arrangement if one pump is down. For the parallel analysis, determine the power required (including viscous corrections, if needed) for the two parallel pump operating conditions.
e. Provide a summary of the recommended pumps for the system including the system capacity when only one of the pumps in the parallel configuration is operating. A recommendation as to which arrangement to use, single main pump or parallel main pumps, is desired.

# Solution
## Part A
The first step for determining whether viscous effects are negligible is to acquire the SSU viscosity. The code block below contains the function for finding the SSU viscosity, given the kinematic viscosity. This was added to the `msu_esd` package, but is shown here for convenience.

In [3]:
def SSU(nu):
    """
    Returns the Saybolt Seconds Universal value of viscosity.

    :param nu: The kinematic viscosity in ft^2/s
    :return: SSU value
    """
    guess = 100
    stoke = 1e-4  # m^2/s
    nu = nu/3.28084**2  # converts to m^2/s

    if nu < 0.2065*stoke:
        return fsolve(lambda ssu: nu - 0.00226*stoke*ssu + 1.95*stoke/ssu, np.array([guess, ]))[0]
    else:
        return fsolve(lambda ssu: nu - 0.0022*stoke*ssu + 1.35*stoke/ssu, np.array([guess, ]))[0]

The kinematic viscosity is,

$$\nu=\frac{\mu}{\rho}=\frac{3.715\,\frac{lbm}{ft\,hr}}{48.05\,\frac{lbm}{ft^3}}=2.148\cdot10^-5\,\frac{ft^2}{s}$$

In [4]:
kin_viscosity = mu/rho
ssu_viscosity = SSU(kin_viscosity)
ssu_viscosity

34.11804944489399

Thus, the SSU value for viscosity is around 34. The flow rate across the main pump is $10.7\,\frac{ft^3}{s}$, which is 4800 gallons per minute. Figure 2 below shows the relationship and procedure for determining the correction factors.

\begin{center}
\includegraphics{images/fig2} \\
Figure 2: Correction Factors
\end{center}

The results show that the correction factors are close to 100% for the head, flow rate, and efficiency because the viscosity of water is low; therefore, **the viscous correction factors are negligible**.